# Bayesian Cosmology

We will attempt to explore a simple cosmological model for the evolution of the universe using some real-world supernova data. This topic is very deep and you can spend an entire degree in physics learning what that goes into cosmology, but we will try and highlight some of the basics and provide links for further reading.  

The cosmology we are going to investigate is a *radiation free* universe, which only has two parameters: 
* $\Omega_{\Lambda}$, also known as dark energy or the [cosmological constant](https://en.wikipedia.org/wiki/Cosmological_constant);
*  $\Omega_M$, also known as [dark matter](https://en.wikipedia.org/wiki/Dark_matter).

We will also assume a prior that the [universe is flat](https://en.wikipedia.org/wiki/Shape_of_the_universe), which essentially just forces the additional constraint at the end that $\Omega_{\Lambda} + \Omega_M = 1$.

For those further interested in the physics, this is the starting point for the [Lambda-CDM model](https://en.wikipedia.org/wiki/Lambda-CDM_model). However, we are just going to assume the physicists know what they are doing, and try calculate posterior distributions over the 2 model parameters using bayesian inference.


# Friedmann Equations


The [Friedmann Equations](https://en.wikipedia.org/wiki/Friedmann_equations) are a set of equations which govern how the expansion of the universe evolves. The Friedmann equation for a flat lambda universe, and therefore of focus to us is
$$
\frac{H^2}{H_0^2} = \Omega_M (1 + z)^3 + (1 - \Omega_{\Lambda} - \Omega_M) (1 + z)^2 + \Omega_{\Lambda}.
$$
$H$ is known as the [Hubble factor](https://en.wikipedia.org/wiki/Hubble%27s_law) and quantifies how quickly the universe is expanding. $z$ is the [redshift](), which states how much light gets distorted as it travels through the universe to reach us from far away stars. $H_0$ is the [Hubble factor today](https://en.wikipedia.org/wiki/Hubble%27s_law#21st_century_measurements), which we will take as a known value $H_0 = 70 \text{km} \text{ s}^{-1} \text{ Mpc}^{-1}$.

Using the Friedmann equation and a bit of nobel-prize winning physics, we can relate the apparent brightness of [supernova]() explosions to their measured redshift. See [Perlmutter97](https://arxiv.org/pdf/astro-ph/9608192.pdf), [Astier05](https://www.aanda.org/articles/aa/pdf/2006/07/aa4185-05.pdf). 
$$
m(z) = 5 \log_{10} (d_L (z ; \Omega_{\Lambda}, \Omega_M) / 10 \text{pc} )
$$

Where the cosmological parametrisation lies in the luminosity distance, $d_L$, given by 
$$
d_L (z ; \Omega_{\Lambda}, \Omega_M) = \frac{c(1+z)}{H_0 \sqrt{|\kappa|}} \mathcal{S} \left ( \sqrt{|\kappa|} \int_0^z [(1+z')^2(1+\Omega_M z') - z'(2+z')\Omega_{\Lambda}]^{-\frac{1}{2}} dz' \right).
$$
Here, $c = 299,792 \text{km/s}$ is the speed of light and
$$
\kappa = 
\begin{cases}
1, &\Omega_{\Lambda} + \Omega_M = 1 \\
1 - \Omega_{\Lambda} - \Omega_M, &\text{otherwise}
\end{cases}
$$
$$
\mathcal{S}(x) = 
\begin{cases}
\sin(x), &\Omega_{\Lambda} + \Omega_M > 1 \\
x, &\Omega_{\Lambda} + \Omega_M = 1\\
\sinh(x), &\Omega_{\Lambda} + \Omega_M < 1
\end{cases}
$$
Now these expressions looks very complicated at first glance, and they are. We dont fully understand them either. However, one of the skills that is important in quant finance is the ability to implement a model that you might not have time or expertise to fully understand. It is important to be able to abstract away the meaning behind equations and continue from there in a pragmatic workplace. 

The above expressions are really just some basic math functions, a piecewise function, and an integral. Implementing this in python shouldn't be too difficult. Take care to make sure all the units are correct and you dont forget to divide by $10\text{pc}$ in your luminosity distance.

![supernova_data](figs/supernova_data.png)
*Supernova brightness data with two cosmological fits overlayed*

Here is the raw supernova brightness data and two fits for different cosmological parameters. We can see how different cosmologies produce a better or worse fit.

# Likelihood Function

Now we have an expression for $m(z ; \Omega_{\Lambda}, \Omega_M)$ we use the supernova dataset from [Astier et al., 2005.](https://www.aanda.org/articles/aa/pdf/2006/07/aa4185-05.pdf) that we have provided in `supernova_data.csv` along with a likelihood function to run a Bayesian analysis. 

The idea is to assume a value for $(\Omega_{\Lambda}, \Omega_M)$ and then calculate the $\chi^2$ of the fit $m(z)$ with our data $D$. The larger the $\chi^2$, the worse the fit is and therefore the lower the likelihood. We can express this by treating the $\chi^2$ value as a boltzmann factor and say our likelihood is proportional to
$$
\mathcal{L}(\Omega_{\Lambda}, \Omega_M) = e^{-\chi^{2} (\Omega_{\Lambda}, \Omega_M)}
$$

The $\chi^2$ is simply the sum of all residuals
$$
\chi^2(\Omega_{\Lambda}, \Omega_M) = \sum_i \left( m_{\text{data}}(z) - m_{\text{model}}(z ; \Omega_{\Lambda}, \Omega_M) \right)^2
$$

Now you have all the maths required to run a 2-dimensional Bayesian inference on a simple cosmological model. You should be able to calculate and plot a posterior distribution which shows the most likely values for our universes dark matter and dark energy densities.

![Cosmological_Posterior](figs/cosmological_posterior.png)

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
import math
import scipy

c = 299792 # speed of light (km/s)
H0 = 70 / 1000000 # hubble constant (km/s/pc)

data = pd.read_csv('supernova_data.csv')
data.sort_values(by = ['z'], inplace=True)

The above graph shows the 2D posterior distribution over the parameters $\Omega_M$ and $\Omega_{\Lambda}$. It is cool to see how it is similar to the graph in the real scientific paper. A value of $\Omega_{\Lambda} > 0$ means that the universe is accelerating, which looks pretty likely from our result!

We can now impose the additional prior belief that the universe is flat, which means that $\Omega_{\Lambda} + \Omega_M = 1$, shown by the black line. With this prior we can plot the posterior distribution along this line and find the most likely parameter values: $\Omega_{\Lambda} = 0.788,  \Omega_M = 0.222$. There are many further statistical tests that can be done to investigate the uncertainty distributions of these parameters, but we'll leave that up to you.

![Flat_Universe](figs/flat_universe.png)